In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, out_channels, bias=True):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(1, out_channels, bias=bias)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)
        self.lin.bias.data.fill_(0)

    def forward(self, x_i, x_j):
        cos_sim = torch.sum(
            torch.mul(F.normalize(x_i), 
                      F.normalize(x_j)), 
            dim=1,
            keepdim=True
        )
        x = self.lin(cos_sim)
        return torch.sigmoid(x)

In [14]:
device=0
log_steps=1
num_layers=2
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [15]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [16]:
adj_t = data.adj_t.to(device)

In [17]:
torch.manual_seed(12345)

In [18]:
model = SAGE(hidden_channels, hidden_channels,
             dropout).to(device)

In [19]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [20]:
predictor = LinkPredictor(1).to(device)

In [21]:
predictor.lin.weight

Parameter containing:
tensor([[0.6750]], device='cuda:0', requires_grad=True)

In [22]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [23]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [24]:
model.reset_parameters()
predictor.reset_parameters()

In [25]:
predictor.lin.weight

Parameter containing:
tensor([[1.]], device='cuda:0', requires_grad=True)

In [26]:
predictor.lin.bias

Parameter containing:
tensor([0.], device='cuda:0', requires_grad=True)

In [27]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [28]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [29]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 1.0796
Epoch: 04, Loss: 1.0026
Epoch: 06, Loss: 0.9407
Epoch: 08, Loss: 0.8846
Epoch: 10, Loss: 0.8337
Epoch: 12, Loss: 0.7881
Epoch: 14, Loss: 0.7460
Epoch: 16, Loss: 0.7075
Epoch: 18, Loss: 0.6727
Epoch: 20, Loss: 0.6415
Epoch: 22, Loss: 0.6130
Epoch: 24, Loss: 0.5862
Epoch: 26, Loss: 0.5619
Epoch: 28, Loss: 0.5390
Epoch: 30, Loss: 0.5176
Epoch: 32, Loss: 0.4978
Epoch: 34, Loss: 0.4790
Epoch: 36, Loss: 0.4618
Epoch: 38, Loss: 0.4456
Epoch: 40, Loss: 0.4306
Epoch: 42, Loss: 0.4164
Epoch: 44, Loss: 0.4030
Epoch: 46, Loss: 0.3905
Epoch: 48, Loss: 0.3789
Epoch: 50, Loss: 0.3676
Epoch: 52, Loss: 0.3571
Epoch: 54, Loss: 0.3471
Epoch: 56, Loss: 0.3375
Epoch: 58, Loss: 0.3284
Epoch: 60, Loss: 0.3203
Epoch: 62, Loss: 0.3122
Epoch: 64, Loss: 0.3045
Epoch: 66, Loss: 0.2964
Epoch: 68, Loss: 0.2895
Epoch: 70, Loss: 0.2822
Epoch: 72, Loss: 0.2755
Epoch: 74, Loss: 0.2695
Epoch: 76, Loss: 0.2630
Epoch: 78, Loss: 0.2575
Epoch: 80, Loss: 0.2518
Epoch: 82, Loss: 0.2466
Epoch: 84, Loss:

In [30]:
predictor.lin.weight

Parameter containing:
tensor([[6.5448]], device='cuda:0', requires_grad=True)

In [31]:
predictor.lin.bias

Parameter containing:
tensor([-3.0384], device='cuda:0', requires_grad=True)

In [32]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
  )
)

In [33]:
emb.weight

Parameter containing:
tensor([[ 0.3296,  0.4399, -0.3005,  ...,  0.0863, -0.0949, -0.2286],
        [ 0.1954,  0.0659,  0.1087,  ..., -0.2149, -0.0219, -0.3255],
        [ 0.2050, -0.0375,  0.0098,  ...,  0.1400,  0.0418, -0.1927],
        ...,
        [ 0.1662, -0.1568,  0.1821,  ..., -0.1473,  0.0136,  0.0193],
        [ 0.2386,  0.2067, -0.0423,  ...,  0.0519, -0.0993, -0.1415],
        [-0.1983, -0.0298,  0.2893,  ...,  0.1076, -0.2450, -0.0300]],
       device='cuda:0', requires_grad=True)

In [34]:
h = model(emb.weight, adj_t)

In [35]:
h

tensor([[-0.2974, -0.2205,  0.1148,  ...,  0.3135, -0.2393,  0.2192],
        [-0.2778, -0.2227, -0.2823,  ...,  0.7176,  0.1552,  0.4166],
        [ 0.6545,  1.1164,  0.4466,  ...,  0.5733,  0.0847, -0.0410],
        ...,
        [ 0.4415, -0.4739, -0.1426,  ..., -0.0886,  0.1222, -0.4141],
        [-0.1508, -0.1032, -0.1301,  ...,  0.4898, -0.3675,  0.3532],
        [-0.4365,  0.5831, -0.1910,  ..., -1.3058,  1.1039, -1.1480]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [36]:
predictor(h[0:2], h[411:413])

tensor([[0.9011],
        [0.1243]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [37]:
predictor(h[0:2], h[2:4])

tensor([[0.0509],
        [0.1749]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [38]:
(h[0] * h[2]).sum()

tensor(0.1524, device='cuda:0', grad_fn=<SumBackward0>)

In [39]:
(h[0] * h[411]).sum()

tensor(6.0183, device='cuda:0', grad_fn=<SumBackward0>)

In [40]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_46_h.npy', h.cpu().detach().numpy())
torch.save(model, '../../models/graphsage_link_pred/46_model.pt')
torch.save(emb, '../../models/graphsage_link_pred/46_embedding.pt')
torch.save(predictor, '../../models/graphsage_link_pred/46_predictor.pt')